# Scratchpad for paper revisions

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import os, sys
root_path = os.path.realpath('../')
sys.path.append(root_path)

import torch
from pathlib import Path

from utils.data import make_dataset
from utils.nnet import get_device

from hebbcl.logger import MetricLogger
from hebbcl.model import Nnet
from hebbcl.trainer import Optimiser, train_model
from hebbcl.parameters import parser
from hebbcl.tuner import HPOTuner

In [ ]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]
dict(sorted(vars(args).items(),key=lambda k: k[0]))


## hyperparameter optimisation
hpo on network trained with fewer episodes

In [4]:
args = parser.parse_args(args=[])
args.n_episodes = 8
dict(sorted(vars(args).items(),key=lambda k: k[0]))

{'centering': True,
 'ctx_avg': True,
 'ctx_avg_alpha': 1,
 'ctx_avg_type': 'ema',
 'ctx_avg_window': 50,
 'ctx_scaling': 2,
 'ctx_w_init': 0.5,
 'ctx_weights': False,
 'cuda': False,
 'gating': 'oja_ctx',
 'hebb_normaliser': 10.0,
 'log_interval': 50,
 'loss_funct': 'reward',
 'lrate_hebb': 0.01,
 'lrate_sgd': 0.01,
 'n_episodes': 8,
 'n_features': 27,
 'n_hidden': 100,
 'n_out': 1,
 'n_runs': 50,
 'perform_hebb': True,
 'perform_sgd': True,
 'save_dir': 'simu1',
 'save_results': True,
 'training_schedule': 'blocked',
 'verbose': True,
 'weight_init': 0.01}

In [5]:
# init tuner
tuner = HPOTuner(args, time_budget=60*30, metric="acc")

2022-06-22 14:14:31,142	INFO services.py:1374 -- View the Ray dashboard at http://127.0.0.1:8265
2022-06-22 14:14:31,964	INFO packaging.py:353 -- Creating a file package for local directory 'D:\DPHIL_02_CONTLEARN\paper\codebase\utils'.
2022-06-22 14:14:31,989	INFO packaging.py:222 -- Pushing file package 'gcs://_ray_pkg_f415459db55ddc24.zip' (0.12MiB) to Ray cluster...
2022-06-22 14:14:31,994	INFO packaging.py:225 -- Successfully pushed file package 'gcs://_ray_pkg_f415459db55ddc24.zip'.
2022-06-22 14:14:32,031	INFO packaging.py:353 -- Creating a file package for local directory 'D:\DPHIL_02_CONTLEARN\paper\codebase\hebbcl'.
2022-06-22 14:14:32,054	INFO packaging.py:222 -- Pushing file package 'gcs://_ray_pkg_3a4ab1b19a98af51.zip' (0.06MiB) to Ray cluster...
2022-06-22 14:14:32,057	INFO packaging.py:225 -- Successfully pushed file package 'gcs://_ray_pkg_3a4ab1b19a98af51.zip'.
2022-06-22 14:14:32,062	INFO packaging.py:353 -- Creating a file package for local directory '../ray_tune/'.
2

(pid=None) 2022-06-22 14:14:34,167	INFO working_dir.py:85 -- Setup working dir for gcs://_ray_pkg_f3330b0ae3f3fee2.zip


In [ ]:
# HPO on blocked with oja (all units)
tuner.tune(n_samples=500)

In [22]:
# tuner.results.sort_values("mean_acc",ascending=False).head(20)
df = tuner.results
df = df[["mean_loss", "mean_acc", "config.lrate_sgd","config.lrate_hebb", "config.ctx_scaling","done"]]
df = df[df["done"]==True]
df = df.drop(columns=["done"])
df = df.dropna()
df = df.sort_values("mean_loss",ascending=True)

df.reset_index()
df.head(15)

,mean_loss,mean_acc,config.lrate_sgd,config.lrate_hebb,config.ctx_scaling
trial_id,,,,,
598d7_00476,-28.396252,1.000,0.092055,0.010667,4
598d7_00020,-27.885231,1.000,0.099934,0.013958,7
598d7_00077,-26.886284,1.000,0.063824,0.016840,6
598d7_00284,-26.722467,1.000,0.059603,0.012163,4
598d7_00112,-26.718370,1.000,0.085659,0.012762,3
598d7_00289,-25.787209,1.000,0.079369,0.027872,5
598d7_00015,-25.626108,0.950,0.077775,0.006568,7
598d7_00477,-24.990105,0.975,0.056290,0.022741,5
598d7_00075,-24.395048,1.000,0.067065,0.017650,5


In [ ]:
tuner.best_cfg

In [8]:
# with open("../results/raytune_oja_ctx_blocked_8episodes.pkl", "wb") as f:
#     pickle.dump(df, f)

### verify results

In [11]:
# obtain params
args = parser.parse_args(args=[])

# set checkpoint directory
save_dir = (
        Path("checkpoints") / "test_allhebb"
    ) 

# get device (gpu/cpu)
args.device = get_device(args.cuda)[0]

# override defaults 
args.n_episodes = 8
args.lrate_hebb = 0.010667
args.lrate_sgd = 0.092055
args.ctx_scaling = 1

dict(sorted(vars(args).items(),key=lambda k: k[0]))
# create dataset 
dataset = make_dataset(args)

# instantiate logger, model and optimiser:
logger = MetricLogger(save_dir)
model = Nnet(args)
optimiser = Optimiser(args)

# send model to device (GPU?)
model = model.to(args.device)


# train model
train_model(args, model, optimiser, dataset, logger)

step 0, loss: task a -0.4537, task b -0.0139 | acc: task a 0.5000, task b 0.4500
... n_a: 3 n_b: 1
step 50, loss: task a -9.4825, task b -0.2499 | acc: task a 1.0000, task b 0.6000
... n_a: 3 n_b: 1
step 100, loss: task a -13.1450, task b -0.0947 | acc: task a 1.0000, task b 0.6000
... n_a: 3 n_b: 0
step 150, loss: task a -14.0822, task b -0.0061 | acc: task a 1.0000, task b 0.5000
... n_a: 6 n_b: 0
step 200, loss: task a -14.4364, task b -0.0197 | acc: task a 1.0000, task b 0.5000
... n_a: 8 n_b: 0
step 250, loss: task a -13.3495, task b -4.8696 | acc: task a 1.0000, task b 0.6500
... n_a: 0 n_b: 0
step 300, loss: task a -8.0196, task b -10.9904 | acc: task a 0.7000, task b 0.9000
... n_a: 0 n_b: 1
step 350, loss: task a -4.5937, task b -13.5154 | acc: task a 0.6000, task b 1.0000
... n_a: 0 n_b: 2
done


In [15]:
print(f"config: lrate_sgd: {args.lrate_sgd:.4f}, lrate_hebb: {args.lrate_hebb:.4f}, context offset: {args.ctx_scaling}")
print(f"terminal accuracy: {logger.acc_total[-1]:.2f}, loss: {logger.losses_total[-1]:.2f}")

config: lrate_sgd: 0.0921, lrate_hebb: 0.0107, context offset: 1
terminal accuracy: 0.80, loss: -18.11


In [21]:
df = tuner.results
